# YOLO frame inference
Notebook walkthrough for running `inference.py` on extracted frames and writing `submission.json`. 
Update the config cell to point at your weights and frame folders, then run the notebook top to bottom.

In [ ]:
import os
import json
import random
import logging
from pathlib import Path
from types import SimpleNamespace

import cv2
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO

from code.predict import (
    draw_bbox,
    list_video_dirs,
    parse_frame_idx,
    process_video,
    setup_logger,
)


## Set seed

In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything()


## Configure paths and thresholds

In [ ]:
WEIGHTS = Path('/code/checkpoint/v8s_640_bs_16_30eps_v3/weights/best.pt')
FRAMES_ROOT = Path('/data/extracted_frames')
OUT_DIR = Path('/result/notebook_run')
IMG_SIZE = 640
CONF = 0.3
IOU = 0.45
DEVICE = '0'  # set to 'cpu' to disable GPU
SAVE_VIS = False  # set True to dump visualizations under OUT_DIR/visualize

OUT_DIR.mkdir(parents=True, exist_ok=True)
VIS_ROOT = OUT_DIR / 'visualize' if SAVE_VIS else None
if VIS_ROOT:
    VIS_ROOT.mkdir(parents=True, exist_ok=True)

logger = setup_logger()
logger.info('weights: %s', WEIGHTS)
logger.info('frames_root: %s', FRAMES_ROOT)


## Load model and inspect data

In [ ]:
if not WEIGHTS.exists():
    raise FileNotFoundError(f'Weights not found: {WEIGHTS}')

logger.info('Loading model...')
model = YOLO(str(WEIGHTS))

videos = list_video_dirs(FRAMES_ROOT)
print(f'Found {len(videos)} video folders under {FRAMES_ROOT}')
if videos:
    print('Sample IDs:', [v.name for v in videos[:5]])

args = SimpleNamespace(
    imgsz=IMG_SIZE,
    conf=CONF,
    iou=IOU,
    device=DEVICE,
    save_vis=SAVE_VIS,
)


## Run inference

In [ ]:
submission = []

for vdir in tqdm(videos, desc='Processing videos'):
    result = process_video(vdir, model, args, VIS_ROOT, logger)
    submission.append(result)

print(f'Completed {len(submission)} items')


## Save submission

In [ ]:
out_json = OUT_DIR / 'submission.json'
with out_json.open('w', encoding='utf-8') as f:
    json.dump(submission, f, ensure_ascii=False, indent=2)

print(f'Wrote results → {out_json}')
if submission:
    print('Preview:', json.dumps(submission[0], ensure_ascii=False, indent=2)[:500])


## (Optional) Inspect a visualization frame

In [ ]:
if SAVE_VIS and VIS_ROOT:
    sample = next(VIS_ROOT.glob('*/*.jpg'), None)
    if sample:
        print(f'Sample visualization: {sample}')
    else:
        print('No visualization files written yet.')
else:
    print('Set SAVE_VIS=True in the config cell to export visualization frames.')
